# 라이브러리 로드 및 필요 함수 정의

In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import json
import cv2
import matplotlib.pyplot as plt

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score 
import time

In [2]:
# score 계산
def accuracy_function(real, pred):                   # https://dacon.io/competitions/official/235870/codeshare/4146
    score = f1_score(real, pred, average='macro')
    return score

# torch model 저장
def model_save(model, score,  path):
    os.makedirs('model', exist_ok=True) # if no path , will make new path and save there
    torch.save({
        'model': model.state_dict(),
        'score': score
    }, path)


# 데이터 로드

In [3]:
sorted(glob('dataset/train/train/*/*.json'))

['dataset/train/train\\10027\\10027.json',
 'dataset/train/train\\10037\\10037.json',
 'dataset/train/train\\10043\\10043.json',
 'dataset/train/train\\10045\\10045.json',
 'dataset/train/train\\10063\\10063.json',
 'dataset/train/train\\10090\\10090.json',
 'dataset/train/train\\10109\\10109.json',
 'dataset/train/train\\10115\\10115.json',
 'dataset/train/train\\10117\\10117.json',
 'dataset/train/train\\10118\\10118.json',
 'dataset/train/train\\10124\\10124.json',
 'dataset/train/train\\10169\\10169.json',
 'dataset/train/train\\10173\\10173.json',
 'dataset/train/train\\10192\\10192.json',
 'dataset/train/train\\10199\\10199.json',
 'dataset/train/train\\10201\\10201.json',
 'dataset/train/train\\10224\\10224.json',
 'dataset/train/train\\10226\\10226.json',
 'dataset/train/train\\10238\\10238.json',
 'dataset/train/train\\10251\\10251.json',
 'dataset/train/train\\10252\\10252.json',
 'dataset/train/train\\10263\\10263.json',
 'dataset/train/train\\10274\\10274.json',
 'dataset/t

In [4]:
# 주어진 훈련 데이터 경로
train_csv = sorted(glob('dataset/train/train/*/*.csv')) 
train_jpg = sorted(glob('dataset/train/train/*/*.jpg'))
train_json = sorted(glob('dataset/train/train/*/*.json'))

crops = []
diseases = []
risks = []
labels = []

# 훈련데이터 로드 (in json)
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample["annotations"]["crop"]
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"

        crops.append(crop)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)
label_unique = sorted(np.unique(labels))
label_unique

['1_00_0',
 '2_00_0',
 '2_a5_2',
 '3_00_0',
 '3_a9_1',
 '3_a9_2',
 '3_a9_3',
 '3_b3_1',
 '3_b6_1',
 '3_b7_1',
 '3_b8_1',
 '4_00_0',
 '5_00_0',
 '5_a7_2',
 '5_b6_1',
 '5_b7_1',
 '5_b8_1',
 '6_00_0',
 '6_a11_1',
 '6_a11_2',
 '6_a12_1',
 '6_a12_2',
 '6_b4_1',
 '6_b4_3',
 '6_b5_1']

In [5]:
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}
label_unique

{'1_00_0': 0,
 '2_00_0': 1,
 '2_a5_2': 2,
 '3_00_0': 3,
 '3_a9_1': 4,
 '3_a9_2': 5,
 '3_a9_3': 6,
 '3_b3_1': 7,
 '3_b6_1': 8,
 '3_b7_1': 9,
 '3_b8_1': 10,
 '4_00_0': 11,
 '5_00_0': 12,
 '5_a7_2': 13,
 '5_b6_1': 14,
 '5_b7_1': 15,
 '5_b8_1': 16,
 '6_00_0': 17,
 '6_a11_1': 18,
 '6_a11_2': 19,
 '6_a12_1': 20,
 '6_a12_2': 21,
 '6_b4_1': 22,
 '6_b4_3': 23,
 '6_b5_1': 24}

In [6]:
labels = [label_unique[k] for k in labels]
labels

[9,
 3,
 3,
 3,
 3,
 9,
 9,
 3,
 11,
 4,
 0,
 11,
 7,
 16,
 9,
 3,
 0,
 3,
 7,
 10,
 16,
 14,
 0,
 11,
 10,
 3,
 8,
 4,
 0,
 3,
 9,
 17,
 17,
 3,
 17,
 2,
 5,
 17,
 2,
 3,
 4,
 0,
 1,
 3,
 3,
 3,
 3,
 13,
 0,
 17,
 0,
 14,
 9,
 0,
 3,
 2,
 2,
 4,
 0,
 3,
 11,
 8,
 3,
 11,
 3,
 11,
 3,
 0,
 17,
 1,
 14,
 0,
 11,
 3,
 14,
 1,
 3,
 11,
 17,
 10,
 17,
 0,
 3,
 17,
 11,
 17,
 0,
 0,
 11,
 3,
 11,
 5,
 9,
 1,
 0,
 17,
 2,
 3,
 17,
 2,
 9,
 2,
 10,
 0,
 4,
 3,
 2,
 7,
 9,
 10,
 9,
 3,
 0,
 13,
 0,
 7,
 0,
 11,
 15,
 7,
 11,
 17,
 17,
 4,
 17,
 0,
 11,
 3,
 11,
 3,
 3,
 9,
 16,
 2,
 0,
 4,
 16,
 11,
 17,
 17,
 11,
 7,
 3,
 11,
 7,
 17,
 11,
 13,
 3,
 0,
 2,
 0,
 3,
 9,
 17,
 17,
 1,
 17,
 14,
 0,
 10,
 6,
 3,
 7,
 0,
 0,
 3,
 11,
 9,
 9,
 0,
 17,
 10,
 14,
 14,
 17,
 11,
 13,
 8,
 4,
 0,
 3,
 17,
 0,
 3,
 17,
 3,
 0,
 0,
 3,
 17,
 8,
 10,
 11,
 3,
 8,
 17,
 17,
 0,
 2,
 3,
 4,
 17,
 3,
 3,
 0,
 0,
 17,
 22,
 10,
 3,
 1,
 5,
 11,
 3,
 3,
 11,
 3,
 14,
 3,
 11,
 13,
 3,
 17,
 0,
 11,
 3,
 17,
 3

# Complete Code

In [7]:
# 주어진 훈련 데이터 경로
train_csv = sorted(glob('dataset/train/train/*/*.csv')) 
train_jpg = sorted(glob('dataset/train/train/*/*.jpg'))
train_json = sorted(glob('dataset/train/train/*/*.json'))

crops = []
diseases = []
risks = []
labels = []

# 훈련데이터 로드
for i in range(len(train_json)):
    with open(train_json[i], 'r') as f:
        sample = json.load(f)
        crop = sample['annotations']['crop']
        disease = sample['annotations']['disease']
        risk = sample['annotations']['risk']
        label=f"{crop}_{disease}_{risk}"
    
        crops.append(crop)
        diseases.append(disease)
        risks.append(risk)
        labels.append(label)

label_unique = sorted(np.unique(labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

labels = [label_unique[k] for k in labels]


# 이미지 로드 함수 / 로드 후 (512, 384)로 사이즈 변환 ==> (원본 사이즈에서 width 384인 샘플이 많아서 (512, 384) 사이즈로 변환)
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 512)) # width , height! 
    return img

# 이미지 로드
imgs = [img_load(k) for k in tqdm(train_jpg)]

100%|██████████| 5767/5767 [00:25<00:00, 224.45it/s]


# 데이터로더와 모델 정의

In [8]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode=='train':
            augmentation = random.randint(0,2)
            if augmentation==1:  #if 1, do horizontal flip 
                img = img[::-1].copy() # width , horizontal flip
            elif augmentation==2: # if 2, do vertical flip
                img = img[:,::-1].copy() # height,  vertical flip
        img = transforms.ToTensor()(img)
        if self.mode=='test':
            return img
        
        label = self.labels[idx]
        return img,label

# 모델 정의

In [9]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=25) # change
        
    def forward(self, x):
        x = self.model(x)
        return x   

# 데이터 분리 및 로더 정의

In [11]:
# KFold
folds = []
k_folds = 5
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

# Start print
print('--------------------------------')

for fold, (train_idx, valid_idx) in enumerate(kf.split(imgs)):
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    folds.append((train_idx, valid_idx))
    print(fold)

fold=0
train_idx, valid_idx = folds[fold]

batch_size = 4
epochs = 15

# Train
train_dataset = Custom_dataset(np.array(imgs)[train_idx], np.array(labels)[train_idx], mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size, pin_memory=True, num_workers=4)

# Validation 
valid_dataset = Custom_dataset(np.array(imgs)[valid_idx], np.array(labels)[valid_idx], mode='valid')
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=batch_size, pin_memory=True, num_workers=4)

--------------------------------
FOLD 0
--------------------------------
0
FOLD 1
--------------------------------
1
FOLD 2
--------------------------------
2
FOLD 3
--------------------------------
3
FOLD 4
--------------------------------
4


# 훈련

In [12]:
device = torch.device('cuda')
model = Network().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    # Training
    model.train()
    for batch in (train_loader):
        # batch -> (inp,lbl)
        optimizer.zero_grad() # zero the parameter gradients
        inputs = torch.tensor(batch[0], dtype=torch.float32, device=device)
        labels = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            outputs = model(inputs)       #  model(inputs)
        loss = criterion(outputs, labels) #  criterion(outputs, labels)

        # see "Typical Mixed O"
        scaler.scale(loss).backward() # loss.backward() 
        scaler.step(optimizer) # optimizer.step()
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += outputs.argmax(1).detach().cpu().numpy().tolist() # torch.max(outputs, 1)
        train_y += labels.detach().cpu().numpy().tolist()
        
    
    train_f1 = accuracy_function(train_y, train_pred)
    
    model.eval()
    valid_loss = 0
    valid_pred=[]
    valid_y=[]

    #Validation
    with torch.no_grad():
        for batch in (valid_loader):
            inputs = torch.tensor(batch[0], dtype=torch.float32, device=device)
            labels = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()/len(valid_loader)
            valid_pred += outputs.argmax(1).detach().cpu().numpy().tolist()
            valid_y += labels.detach().cpu().numpy().tolist()
        valid_f1 = accuracy_function(valid_y, valid_pred)
    if valid_f1>=best:
        best=valid_f1
        model_save(model, valid_f1, f'model/eff-b0.pth')
    # visualize the training process with the evaluation metric
    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'VALID    loss : {valid_loss:.5f}    f1 : {valid_f1:.5f}    best : {best:.5f}')